### 框架干的最厉害的一件事就是帮我们把返向传播全部计算好了

In [48]:
import torch

需要求导的，可以手动定义：

In [49]:
#方法1
x = torch.randn(3,4,requires_grad=True)
x

tensor([[-0.6812,  0.1245,  0.4627, -0.5860],
        [ 1.2594, -0.4262,  0.9005, -0.4189],
        [ 0.6924, -1.0704,  0.3465, -0.8244]], requires_grad=True)

In [50]:
#方法2
x = torch.randn(3,4)
x.requires_grad=True
x

tensor([[-1.0556,  1.2333, -0.9068,  1.1550],
        [-0.3289, -1.9466,  0.1828, -1.7511],
        [-0.4664,  0.5741,  0.9633, -0.3505]], requires_grad=True)

In [51]:
b = torch.randn(3,4,requires_grad=True)

In [52]:
t = x + b

In [53]:
y = t.sum()
y

tensor(-2.1930, grad_fn=<SumBackward0>)

In [54]:
y.backward()

In [55]:
b.grad

tensor([[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.]])

虽然没有指定t的requires_grad但是需要用到它，也会默认的

In [9]:
x.requires_grad, b.requires_grad, t.requires_grad

(True, True, True)

### 举个例子看一下：

![title](./img/2.png)

In [57]:
#计算流程
x = torch.rand(1)
b = torch.rand(1, requires_grad = True)
w = torch.rand(1, requires_grad = True)
y = w * x 
z = y + b 

In [58]:
x.requires_grad, b.requires_grad, w.requires_grad, y.requires_grad#注意y也是需要的

(False, True, True, True)

In [59]:
x.is_leaf, w.is_leaf, b.is_leaf, y.is_leaf, z.is_leaf

(True, True, True, False, False)

返向传播计算

In [80]:
z.backward(retain_graph=True)#如果不清空会累加起来

In [81]:
w.grad

tensor([0.2456])

In [82]:
b.grad

tensor([8.])

### 做一个线性回归试试水

构造一组输入数据X和其对应的标签y

In [38]:
x_values = [i for i in range(11)]
x_train = np.array(x_values, dtype=np.float32)
x_train = x_train.reshape(-1, 1)
x_train.shape

(11, 1)

In [39]:
y_values = [2*i + 1 for i in x_values]
y_train = np.array(y_values, dtype=np.float32)
y_train = y_train.reshape(-1, 1)
y_train.shape

(11, 1)

In [40]:
import torch
import torch.nn as nn

### 线性回归模型
- 其实线性回归就是一个不加激活函数的全连接层

In [83]:
class LinearRegressionModel(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(LinearRegressionModel, self).__init__()
        self.linear = nn.Linear(input_dim, output_dim)  

    def forward(self, x):
        out = self.linear(x)
        return out

In [84]:
input_dim = 1
output_dim = 1

model = LinearRegressionModel(input_dim, output_dim)

In [85]:
model

LinearRegressionModel(
  (linear): Linear(in_features=1, out_features=1, bias=True)
)

指定好参数和损失函数

In [43]:
epochs = 1000
learning_rate = 0.01
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
criterion = nn.MSELoss()

训练模型

In [44]:
for epoch in range(epochs):
    epoch += 1
    # 注意转行成tensor
    inputs = torch.from_numpy(x_train)
    labels = torch.from_numpy(y_train)

    # 梯度要清零每一次迭代
    optimizer.zero_grad() 

    # 前向传播
    outputs = model(inputs)

    # 计算损失
    loss = criterion(outputs, labels)

    # 返向传播
    loss.backward()

    # 更新权重参数
    optimizer.step()
    if epoch % 50 == 0:
        print('epoch {}, loss {}'.format(epoch, loss.item()))

epoch 50, loss 0.008274978958070278
epoch 100, loss 0.004719749093055725
epoch 150, loss 0.0026919401716440916
epoch 200, loss 0.0015353981871157885
epoch 250, loss 0.0008757374598644674
epoch 300, loss 0.0004994918708689511
epoch 350, loss 0.00028488683165051043
epoch 400, loss 0.00016248888277914375
epoch 450, loss 9.267879067920148e-05
epoch 500, loss 5.285888255457394e-05
epoch 550, loss 3.015184120158665e-05
epoch 600, loss 1.7196194676216692e-05
epoch 650, loss 9.807815331441816e-06
epoch 700, loss 5.593081368715502e-06
epoch 750, loss 3.1907097763905767e-06
epoch 800, loss 1.8201473039880511e-06
epoch 850, loss 1.0386996791567071e-06
epoch 900, loss 5.916471650380117e-07
epoch 950, loss 3.374661332600226e-07
epoch 1000, loss 1.925896526699944e-07


### 测试模型预测结果

In [45]:
predicted = model(torch.from_numpy(x_train).requires_grad_()).data.numpy()
predicted

array([[ 0.99918383],
       [ 2.9993014 ],
       [ 4.9994187 ],
       [ 6.9995365 ],
       [ 8.999654  ],
       [10.999771  ],
       [12.999889  ],
       [15.000007  ],
       [17.000124  ],
       [19.000242  ],
       [21.000359  ]], dtype=float32)

### 模型的保存与读取

In [22]:
torch.save(model.state_dict(), 'model.pkl')

In [23]:
model.load_state_dict(torch.load('model.pkl'))

<All keys matched successfully>

### 使用GPU进行训练
- 只需要把数据和模型传入到cuda里面就可以了

In [27]:
import torch
import torch.nn as nn
import numpy as np


class LinearRegressionModel(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(LinearRegressionModel, self).__init__()
        self.linear = nn.Linear(input_dim, output_dim)  

    def forward(self, x):
        out = self.linear(x)
        return out

input_dim = 1
output_dim = 1

model = LinearRegressionModel(input_dim, output_dim)


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)


criterion = nn.MSELoss()


learning_rate = 0.01

optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 1000
for epoch in range(epochs):
    epoch += 1
    inputs = torch.from_numpy(x_train).to(device)
    labels = torch.from_numpy(y_train).to(device)

    optimizer.zero_grad() 

    outputs = model(inputs)

    loss = criterion(outputs, labels)

    loss.backward()

    optimizer.step()

    if epoch % 50 == 0:
        print('epoch {}, loss {}'.format(epoch, loss.item()))

epoch 50, loss 0.011100251227617264
epoch 100, loss 0.006331132724881172
epoch 150, loss 0.003611058695241809
epoch 200, loss 0.0020596047397702932
epoch 250, loss 0.0011747264070436358
epoch 300, loss 0.0006700288504362106
epoch 350, loss 0.00038215285167098045
epoch 400, loss 0.00021796672081109136
epoch 450, loss 0.00012431896175257862
epoch 500, loss 7.090995495673269e-05
epoch 550, loss 4.044298475491814e-05
epoch 600, loss 2.3066799258231185e-05
epoch 650, loss 1.3156819477444515e-05
epoch 700, loss 7.503344477299834e-06
epoch 750, loss 4.279831500753062e-06
epoch 800, loss 2.4414177914877655e-06
epoch 850, loss 1.3924694712841301e-06
epoch 900, loss 7.945647553242452e-07
epoch 950, loss 4.530382398115762e-07
epoch 1000, loss 2.5830334493548435e-07
